# 예측 1

라이브러리

In [305]:
# 공통
%matplotlib inline
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [306]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('전체.csv', encoding='cp949')
data = data[:-365]

data = data.drop(['price'], axis = 1)
data = data.drop(['전체_temp'], axis = 1)
data = data.drop(['전체_hum'], axis = 1)

data = data.drop(['강원_temp'], axis = 1)
data = data.drop(['경기_temp'], axis = 1)
data = data.drop(['경남_temp'], axis = 1)
data = data.drop(['경북_temp'], axis = 1)
data = data.drop(['광주_temp'], axis = 1)
data = data.drop(['대구_temp'], axis = 1)
data = data.drop(['대전_temp'], axis = 1)
data = data.drop(['부산_temp'], axis = 1)
data = data.drop(['서울_temp'], axis = 1)
data = data.drop(['세종_temp'], axis = 1)
data = data.drop(['울산_temp'], axis = 1)
data = data.drop(['충남_temp'], axis = 1)
data = data.drop(['충북_temp'], axis = 1)
data = data.drop(['전남_temp'], axis = 1)
data = data.drop(['전북_temp'], axis = 1)
data = data.drop(['제주_temp'], axis = 1)
data = data.drop(['인천_temp'], axis = 1)

data = data.drop(['강원_hum'], axis = 1)
data = data.drop(['경기_hum'], axis = 1)
data = data.drop(['경남_hum'], axis = 1)
data = data.drop(['경북_hum'], axis = 1)
data = data.drop(['광주_hum'], axis = 1)
data = data.drop(['대구_hum'], axis = 1)
data = data.drop(['대전_hum'], axis = 1)
data = data.drop(['부산_hum'], axis = 1)
# data = data.drop(['서울_hum'], axis = 1)
data = data.drop(['세종_hum'], axis = 1)
data = data.drop(['울산_hum'], axis = 1)
data = data.drop(['충남_hum'], axis = 1)
data = data.drop(['충북_hum'], axis = 1)
data = data.drop(['전남_hum'], axis = 1)
data = data.drop(['전북_hum'], axis = 1)
data = data.drop(['제주_hum'], axis = 1)
data = data.drop(['인천_hum'], axis = 1)
    
data = data.set_index('date')
data = data.fillna(method='ffill')
data.tail()

,서울_hum
date,
2021-12-27,60.9
2021-12-28,73.8
2021-12-29,72.9
2021-12-30,48.5
2021-12-31,35.9


In [307]:
# 인덱스 지정(?)

# strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# strftime은 날짜와 시간(datetime)을 문자열로 출력
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
date_list = [start + relativedelta(days = x) for x in range(0, len(data)) ]

data['index'] = date_list

# set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
data.set_index( ['index'], inplace=True) 

# index 출력 X
data.index.name=None
data.tail()

,서울_hum
2021-12-27,60.9
2021-12-28,73.8
2021-12-29,72.9
2021-12-30,48.5
2021-12-31,35.9


In [308]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    data['서울_hum'],
    order = (1,0,0),
    seasonal_order = (2,0,1,7),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5414      0.047     11.570      0.000       0.450       0.633
ar.S.L7        0.8632      0.056     15.359      0.000       0.753       0.973
ar.S.L14       0.1372      0.056      2.437      0.015       0.027       0.248
ma.S.L7       -0.8603      0.041    -21.126      0.000      -0.940      -0.781
sigma2       117.0446      8.092     14.464      0.000     101.184     132.905


In [309]:
pred = results.get_prediction(
    start = pd.to_datetime('2021-01-01'),
    # 동적 회귀(?)
    dynamic = False
)

In [310]:
y_forecasted = pred.predicted_mean
y_truth = data['서울_hum']['2021-01-01' : ]

# Compute the Mean Square Error: 평균 제곱 오차 계산
mse = (( y_forecasted - y_truth ) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format( round(mse, 2) ) ) 

The Mean Squared Error of our forecasts is 148.67


In [311]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 365 )

In [312]:
forecast = pred_uc.predicted_mean
forecast

2022-01-01    49.809882
2022-01-02    54.665874
2022-01-03    62.273570
2022-01-04    69.550517
2022-01-05    66.047445
                ...    
2022-12-27    73.299736
2022-12-28    69.152070
2022-12-29    65.484130
2022-12-30    63.044781
2022-12-31    62.202614
Freq: D, Name: predicted_mean, Length: 365, dtype: float64

In [313]:
# 예측 데이터 csv로 저장
forecast.to_csv('변수X_2022.csv', index=False)